Імпорти

In [1]:
import pandas as pd
import urllib.request
from datetime import datetime
from os import walk
import csv

Змінні

In [2]:
DIRECTORY = 'CSV_Files/'
URL = ('https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={'
       '}&year1=1981&year2=2020&type=Mean')
INDEXES = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12, 16: 13,
           17: 14, 18: 15, 19: 16, 21: 17, 22: 18, 24: 1, 25: 2, 26: 7, 27: 5}

Завантаження файлів

In [3]:
for province_id in range(1, 29):
    url = URL.format(province_id)
    wp = urllib.request.urlopen(url)
    text = wp.read()
    now = datetime.now()
    date_and_time_time = now.strftime("%d-%m-%Y_%H-%M-%S")
    out = open(DIRECTORY + 'NOAA_ID' + str(province_id) + '_' + date_and_time_time + '.csv', 'wb')
    out.write(text)
    out.close()
    # print('File for ' + str(province_id))
print('Success')

Success


Створення фрейму з файлів у папці

In [5]:
headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
i = 1
df_list = []
for (dirpath, dirnames, filenames) in walk(DIRECTORY):
    for filename in filenames:
        df = pd.read_csv(DIRECTORY + filename, header=1, names=headers)
        df.at[0, 'Year'] =  df.at[0, 'Year'][9:]
        df = df.drop(df.index[-1])
        df = df.drop(df.loc[df['VHI'] == -1].index)
        df = df.drop('empty', axis=1)
        df['area'] = i
        i += 1
        df_list.append(df)
df = pd.concat(df_list)

Зміна індексів

In [6]:
df.replace({'area': INDEXES}, inplace=True)

Перша процедура

In [7]:
def procedure_1(df, year, province_n):
    df_new = df[(df.area == province_n) & (df.Year.astype(str) == str(year))]['VHI']
    print('For ' + str(year) + ' and the province ' + str(province_n) + ':')
    print('Min: ' + str(df_new.min()))
    print('Max: ' + str(df_new.max()))

Друга процедура

In [8]:
def procedure_2(df, province_n, percent=50):
    df_drought = df[(df["area"] == province_n) & (df.VHI <= percent)]
    list_from_ds = list(df_drought.Year)
    years = []
    [years.append(item) for item in list_from_ds if item not in years]
    print('For the province ' + str(province_n) + ' extreme drought is in:')
    print('Years: ' + ' '.join(years))

Третя процедура

In [9]:
def procedure_3(df, province_n):
    df_drought = df[(df["area"] == province_n) & (df.VHI <= 35) & (df.VHI > 15)]
    list_from_ds = list(df_drought.Year)
    years = []
    [years.append(item) for item in list_from_ds if item not in years]
    print('For the province ' + str(province_n) + ' moderate drought is in:')
    print('Years: ' + ' '.join(years))

Завантаження всіх файлів

In [10]:
procedure_1(df, 2000, 3)

For 2000 and the province 3:
Min: 22.92
Max: 74.06


In [ ]:
procedure_2(df, 3)

In [ ]:
procedure_3(df, 3)